In [1]:
import numpy as np
import kv_lite as kv

import time

In [6]:
expr_ab = kv.Symbol('a') * kv.Symbol('b')
sym_c   = kv.Symbol('c')
expr_ab_t0 = expr_ab.set_stamp(0) + sym_c
expr_ab_tm1 = expr_ab.set_stamp(-1) + sym_c
expr_ab_tm2 = expr_ab.set_stamp(-2) + sym_c

l = kv.VectorizedLayout(kv.array(expr_ab_t0 + expr_ab_tm2),
                        [0, 1, 2, 3, 4],
                        list(expr_ab.symbols | {sym_c}),
                        diff_symbols={kv.Symbol('a'), kv.Symbol('c')}, order=0)
l.eval_expr(np.random.random((5, 2)), 4, series_pad_values=np.random.random((2, 2)))
l.eval_J(np.random.random((5, 2)), 4, series_pad_values=np.random.random((2, 2)))

array([[0.        , 0.        , 2.        ],
       [1.        , 0.        , 2.        ],
       [1.        , 1.        , 0.23211218],
       [2.        , 0.        , 2.        ],
       [2.        , 1.        , 0.52090147],
       [2.        , 2.        , 0.53897829],
       [3.        , 0.        , 2.        ],
       [3.        , 2.        , 0.23211218],
       [3.        , 3.        , 0.55165352],
       [4.        , 0.        , 2.        ],
       [4.        , 3.        , 0.53897829],
       [4.        , 4.        , 0.24647567]])

In [7]:
bla = kv.VectorizedLayout(kv.array(expr_ab_t0),
                                              [2],
                                              list(expr_ab.symbols | {sym_c}),
                                              diff_symbols={kv.Symbol('a'), kv.Symbol('c')}, order=0)

m = kv.MacroLayout(dict(first=l,
                   second=bla))
print(m.in_dim, m.out_dim)
m.eval_all(np.ones(m.in_dim))

11 6


(array([4., 4., 4., 4., 4., 2.]),
 array([[0., 0., 2.],
        [1., 0., 2.],
        [1., 1., 1.],
        [2., 0., 2.],
        [2., 1., 1.],
        [2., 2., 1.],
        [3., 0., 2.],
        [3., 2., 1.],
        [3., 3., 1.],
        [4., 0., 2.],
        [4., 3., 1.],
        [4., 4., 1.],
        [5., 0., 1.],
        [5., 3., 1.]]))

In [12]:
solver = kv.RAI_NLPSolver(dict(first=(kv.SolverObjectives.sos, l), second=(kv.SolverObjectives.eq, bla)),
                          constants=np.arange(5, dtype=float))
shared, series, ret = solver.solve(np.random.random(6))
print(solver.shared_symbols)
print(shared)
print(solver.series_symbols)
print(series)
print(ret.sos, ret.eq, ret.feasible)
print(solver.report(ret.x))

[KV(c)]
[-0.43015503]
[KV(b) KV(a)]
[[ 0.         -5.54011093]
 [ 1.          3.70878719]
 [ 2.          0.2154679 ]
 [ 3.         -2.99990431]
 [ 4.          0.85486746]]
55.81669165669523 0.0007807748970946604 True
{'first': array([[-0.86031007],
       [ 2.84847712],
       [-0.42937426],
       [-6.15123581],
       [ 2.9900956 ]]), 'second': array([[0.00078077]])}


In [6]:
km = kv.Model()

with open('/home/adrian/lh_ws/src/llm_stacking/urdf/nextage.urdf', 'r') as f:
    robot = kv.urdf.load_urdf(km, f.read())


In [7]:
robot.links.keys()

dict_keys(['world', 'base_link', 'nextage_base', 'CAMERA_HEAD_L_Link', 'CAMERA_HEAD_R_Link', 'WAIST', 'CHEST_JOINT0_Link', 'HEAD_JOINT0_Link', 'HEAD_JOINT1_Link', 'LARM_JOINT0_Link', 'LARM_JOINT1_Link', 'LARM_JOINT2_Link', 'LARM_JOINT3_Link', 'LARM_JOINT4_Link', 'LARM_JOINT5_Link', 'RARM_JOINT0_Link', 'RARM_JOINT1_Link', 'RARM_JOINT2_Link', 'RARM_JOINT3_Link', 'RARM_JOINT4_Link', 'RARM_JOINT5_Link', 'LARM_ft_link_base', 'LARM_ft_Sensor_Link', 'LARM_ft_link', 'RARM_ft_link_base', 'RARM_ft_Sensor_Link', 'RARM_ft_link', 'LARM_changer_link_base', 'LARM_changer_link', 'RARM_changer_link_base', 'RARM_changer_link', 'RARM_robotiq_arg2f_link', 'RARM_left_outer_knuckle', 'RARM_left_outer_finger', 'RARM_left_inner_finger', 'RARM_left_inner_knuckle', 'RARM_right_outer_knuckle', 'RARM_right_outer_finger', 'RARM_right_inner_finger', 'RARM_right_inner_knuckle', 'RARM_gripper_dummy_link', 'RARM_grasp_point', 'LARM_END_EFFECTOR', 'LARM_END_EFFECTOR_finger', 'LARM_END_EFFECTOR_grasp', 'CAMERA_HEAD_Link

In [8]:
w_T_ee = robot.get_fk('RARM_grasp_point', robot.root_link).transform

In [9]:
w_P_ee = w_T_ee[:3, 3]

ee_syms = w_P_ee.ordered_symbols

JD_w_P_ee = w_P_ee.jacobian(ee_syms)
J_coords, J_w_P_ee = JD_w_P_ee.to_coo()
JD_w_P_ee.set_symbol_order(ee_syms)

In [10]:
w_P_ee_t0  = w_P_ee.set_stamp(0)
w_P_ee_tm1 = w_P_ee.set_stamp(-1)
print(w_P_ee_t0.ordered_symbols)
print(w_P_ee_tm1.ordered_symbols)

[KV(nextage__RARM_JOINT1__position__t0), KV(nextage__RARM_JOINT3__position__t0), KV(nextage__CHEST_JOINT0__position__t0), KV(nextage__RARM_JOINT5__position__t0), KV(nextage__RARM_JOINT4__position__t0), KV(nextage__RARM_JOINT0__position__t0), KV(nextage__RARM_JOINT2__position__t0)]
[KV(nextage__RARM_JOINT2__position__t-1), KV(nextage__RARM_JOINT1__position__t-1), KV(nextage__RARM_JOINT3__position__t-1), KV(nextage__RARM_JOINT5__position__t-1), KV(nextage__RARM_JOINT4__position__t-1), KV(nextage__CHEST_JOINT0__position__t-1), KV(nextage__RARM_JOINT0__position__t-1)]


In [11]:
time_series_symbols = [[kv.Symbol(f'{s}_t{t}') for s in ee_syms] for t in range(1000)]
time_series_symbols

[[KV(nextage__RARM_JOINT2__position_t0),
  KV(nextage__RARM_JOINT1__position_t0),
  KV(nextage__RARM_JOINT5__position_t0),
  KV(nextage__RARM_JOINT0__position_t0),
  KV(nextage__RARM_JOINT4__position_t0),
  KV(nextage__CHEST_JOINT0__position_t0),
  KV(nextage__RARM_JOINT3__position_t0)],
 [KV(nextage__RARM_JOINT2__position_t1),
  KV(nextage__RARM_JOINT1__position_t1),
  KV(nextage__RARM_JOINT5__position_t1),
  KV(nextage__RARM_JOINT0__position_t1),
  KV(nextage__RARM_JOINT4__position_t1),
  KV(nextage__CHEST_JOINT0__position_t1),
  KV(nextage__RARM_JOINT3__position_t1)],
 [KV(nextage__RARM_JOINT2__position_t2),
  KV(nextage__RARM_JOINT1__position_t2),
  KV(nextage__RARM_JOINT5__position_t2),
  KV(nextage__RARM_JOINT0__position_t2),
  KV(nextage__RARM_JOINT4__position_t2),
  KV(nextage__CHEST_JOINT0__position_t2),
  KV(nextage__RARM_JOINT3__position_t2)],
 [KV(nextage__RARM_JOINT2__position_t3),
  KV(nextage__RARM_JOINT1__position_t3),
  KV(nextage__RARM_JOINT5__position_t3),
  KV(nexta

In [12]:
j_offset = np.array([len(w_P_ee), len(ee_syms)])

coord_stack = np.vstack([J_coords + t*j_offset for t in range(len(time_series_symbols))])
J_stack     = kv.hstack([J_w_P_ee.substitute(dict(zip(ee_syms, t_syms))) for t_syms in time_series_symbols])
t_series_vars = kv.array(time_series_symbols)

In [13]:
J_stack.set_symbol_order(t_series_vars.flatten())


In [14]:
print('Big eval')
for x in range(50):
    q = np.random.random(t_series_vars.shape)
    start = time.time_ns()
    J_stack(q.flatten())
    time_samples.append(time.time_ns() - start)
print(f'Eval in milliseconds: {np.mean(time_samples)*1e-6} {np.std(time_samples)*1e-6}')

Big eval


NameError: name 'time_samples' is not defined

In [16]:
print('Batched eval')
time_samples = []
for x in range(50):
    q = np.random.random(t_series_vars.shape)
    start = time.time_ns()
    J_w_P_ee(q)
    time_samples.append(time.time_ns() - start)
print(f'Eval in milliseconds: {np.mean(time_samples)*1e-6} {np.std(time_samples)*1e-6}')

Batched eval
Eval in milliseconds: 5.18271666 1.2738920806583518


In [17]:
print('Batched eval')
time_samples = []
for x in range(50):
    q = np.random.random(t_series_vars.shape)
    start = time.time_ns()
    JD_w_P_ee(q)
    time_samples.append(time.time_ns() - start)
print(f'Eval in milliseconds: {np.mean(time_samples)*1e-6} {np.std(time_samples)*1e-6}')

Batched eval
Eval in milliseconds: 5.20472294 1.6120580061838894


In [18]:
import re

time_re = re.compile(r'.+_t(\d+)')

print(time_re.match('_t12'))
print(time_re.match('abc_t12').groups())
print(time_re.match('abc_t'))
print(time_re.match('abc_t12842_t05').groups())

None
('12',)
None
('05',)
